# 서울시 범죄현황 통계자료 분석 및 시각화
<br>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
from matplotlib import font_manager, rc

<br>

#### 1. 데이터 입력 및 데이터 전처리

In [ ]:
# 서울시 관서별 5대 범죄 발생 & 검거 현황 @ data.go.kr
# 원본 데이터 및 전처리 작업을 위한 파이썬 코드 @ folder named [ Original data source & data preprocessing (5대범죄 & 인구수) ] 

df = pd.read_excel('관서별 5대범죄 발생 및 검거.xlsx') # 엑셀 파일 읽기?
df.head()

#### 경찰서를 구별로 정리하기

In [ ]:
# 서울시 경찰청의 소속 구 @ https://goo.gl/MQSqXX
police_to_gu = {'서대문서': '서대문구', '수서서': '강남구', '강서서': '강서구', '서초서': '서초구',
                '서부서': '은평구', '중부서': '중구', '종로서': '종로구', '남대문서': '중구',
                '혜화서': '종로구', '용산서': '용산구', '성북서': '성북구', '동대문서': '동대문구',
                '마포서': '마포구', '영등포서': '영등포구', '성동서': '성동구', '동작서': '동작구',
                '광진서': '광진구', '강북서': '강북구', '금천서': '금천구', '중랑서': '중랑구',
                '강남서': '강남구', '관악서': '관악구', '강동서': '강동구', '종암서': '성북구', 
                '구로서': '구로구', '양천서': '양천구', '송파서': '송파구', '노원서': '노원구', 
                '방배서': '서초구', '은평서': '은평구', '도봉서': '도봉구'}

# dict[칼럼명].apply(칼럼 내 데이터마다 적용할 함수)
# dict.get(key)는 value 를 return
df['구별'] = df['관서명'].apply(lambda x: police_to_gu.get(x, '구 없음')) 
df.head()

In [ ]:
# 관서별 데이터를 구별 데이터로 변경 (index : 관서 이름 -> 구 이름, column은 자동으로 오름차순 정렬됨)
# 같은 구의 경우에는 sum 을 적용 

gu_df = pd.pivot_table(df, index='구별', aggfunc=np.sum)
gu_df

In [ ]:
gu_df = gu_df.drop(['구 없음']) # df.drop([row]) : 해당 행 데이터를 drop == DB 에서 특정 데이터를 drop 하는 것과 동일
gu_df

#### 범죄별로 검거율 계산하기

In [ ]:
# 발생건수 대비 검거건수 -> 검거율 데이터 column을 범죄별로 생성

gu_df['강간검거율'] = gu_df['강간(검거)']/gu_df['강간(발생)']*100
gu_df['강도검거율'] = gu_df['강도(검거)']/gu_df['강도(발생)']*100
gu_df['살인검거율'] = gu_df['살인(검거)']/gu_df['살인(발생)']*100
gu_df['절도검거율'] = gu_df['절도(검거)']/gu_df['절도(발생)']*100
gu_df['폭력검거율'] = gu_df['폭력(검거)']/gu_df['폭력(발생)']*100
gu_df['검거율'] = gu_df['소계(검거)']/gu_df['소계(발생)']*100
gu_df.head()

In [ ]:
# 필요없는 column 지우기 (범죄별 발생 건수와 검거율만 남긴다)

# df.drop(['row']) : 해당 행 데이터를 drop 
# del df['column'] : 해당 열 데이터를 drop

del gu_df['강간(검거)']
del gu_df['강도(검거)']
del gu_df['살인(검거)']
del gu_df['절도(검거)']
del gu_df['폭력(검거)']
del gu_df['소계(발생)']
del gu_df['소계(검거)']

gu_df.head()

In [ ]:
# 발생건수는 2016이고, 그 전에 발생한 범죄에 대한 검거가 2016에 이뤄지면 검거수에 반영된 것
# if 문을 쓸 필요 없이 바로 boolean 체크 후 값 대입 적용이 가능 

gu_df[gu_df[['강간검거율', '강도검거율', '살인검거율', '절도검거율', '폭력검거율']] > 100] = 100
gu_df.head(10)

In [ ]:
# 위 방법과 유사한 방식으로 Filtering 적용이 가능함

# 살인사건 발생 건수가 7건이 넘는 지역구를 추려내려면?
gu_df[ gu_df['살인(발생)'] > 7 ]

In [ ]:
# 살인사건 발생 건수가 7건이 넘고(and) 폭력사건 발생 건수가 2000건을 넘는 지역구를 추려내려면?
gu_df[ (gu_df['살인(발생)'] > 7) & (gu_df['폭력(발생)'] > 2000) ] 

In [ ]:
# 살인사건 발생 건수가 7건이 넘거나(or) 폭력사건 발생 건수가 2000건이 넘는 지역구를 추려내려면?
gu_df[ (gu_df['살인(발생)'] > 7) | (gu_df['폭력(발생)'] > 2000) ] # Shift + 키보드 ₩

In [ ]:
# 살인사건 발생 건수가 5건을 넘지 않는(not) 지역구를 추려내려면?
gu_df[ ~(gu_df['살인(발생)'] > 5) ] # not의 비트 논리 연산자는 "~"입니다.

In [ ]:
# 도봉구의 경우 2020년 살인 발생 & 검거 건수가 0이므로 
# 검거율 계산 시 0으로 나누는 계산으로 인해 '살인검거율'의 값이 NaN(Not a Number, 결측치의 일종)이 됨

gu_df[ gu_df['살인(발생)'] == 0 ] # 살인사건 발생 건수가 0건인 지역구를 추려내려면?

In [ ]:
# '살인검거율' 열의 결측치를 100으로 채워주기 (결측치가 채워진 열을 기존 열에 덮어써줘야 하는 것에 유의)

gu_df['살인검거율'] = gu_df['살인검거율'].fillna(100) # 결측치(N/A)의 값을 채워주다(fill)

In [ ]:
gu_df[ gu_df['살인(발생)'] == 0 ] # 살인사건 발생 건수가 0건인 지역구를 추려내려면?

In [ ]:
gu_df.rename(columns = {'강간(발생)':'강간',
                        '강도(발생)':'강도',
                        '살인(발생)':'살인',
                        '절도(발생)':'절도',
                        '폭력(발생)':'폭력'}, inplace=True) # inplace 옵션 == 덮어쓰기 여부

gu_df.head()

#### 인구 데이터 merge 하기 

In [ ]:
# CSV 는 어떻게 읽을까요? (엑셀과 유사)

popul_df = pd.read_csv('pop_kor.csv', encoding='utf-8') # read_csv 는 encoding 옵션을 직접 지정해줄 수 있습니다. (utf-8, euc-kr, cp949)
popul_df.head()

In [ ]:
# 구별 index 를 기준으로 merge를 할 것이므로, index 를 세팅해주기

popul_df = pd.read_csv('pop_kor.csv', encoding='utf-8', index_col='구별')
popul_df.head()

# 아래와 같이 먼저 read_csv()로 읽어들이고 .set_index()를 적용할 수도 있습니다.
# popul_df = pd.read_csv('pop_kor.csv', encoding='utf-8').set_index('구별')

In [ ]:
gu_df = gu_df.join(popul_df) # df1.join(df2) : df1 의 index를 기준으로 df2 의 index 중 매칭되는 값을 매김
gu_df.head()

<br>

#### 2. 데이터 살펴보기 (Data exploration)

In [ ]:
# 검거율 기준으로 오름차순 정렬하기

gu_df.sort_values(by='검거율', ascending=False, inplace=True) # ascending=False : 내림차순, inplace=True : 덮어쓰기
gu_df.head()

#### 범죄별 발생 건수 정규화하기 (범죄별로 가장 많이 발생한 구가 1 == 100%)

In [ ]:
# 5대 범죄별 수치를 해당 범죄별 최대값으로 나눠줌 

target_col = ['강간', '강도', '살인', '절도', '폭력']
weight_col = gu_df[target_col].max()
weight_col

In [ ]:
crime_count_norm = gu_df[target_col]/weight_col
crime_count_norm

#### 한글 데이터 시각화를 위한 준비

In [ ]:
# jupyter notebook 내에 figure를 보여주기
%matplotlib inline 

# matplotlib의 한글문제를 해결
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
# font_name
rc('font', family=font_name)

#### 구별 살인 발생 순위 살펴보기

In [ ]:
sns.heatmap(crime_count_norm.sort_values(by='살인', ascending=False)) # ascending=False : 내림차순

In [ ]:
# 몇 가지 옵션으로 더 내용을 확인하기 편하도록 수정하기

# 전체 figure 의 사이즈를 조정
plt.figure(figsize = (10,10))

# annot : 셀 내에 수치 입력 여부
# fmt : 셀 내 입력될 수치의 format (f == float)
# linewidths : 셀 간 이격거리 (하얀 부분, 내부 테두리)
# cmap : matplotlib colormap @ https://goo.gl/YWpBES
sns.heatmap(crime_count_norm.sort_values(by='살인', ascending=False), annot=True, fmt='f', linewidths=.5, cmap='Reds')

plt.title('범죄 발생(살인발생으로 정렬) - 각 항목별 최대값으로 나눠 정규화')
plt.show()

In [ ]:
# 살인 대신 절도 기준으로 살펴보기

plt.figure(figsize = (10,10))
sns.heatmap(crime_count_norm.sort_values(by='절도', ascending=False), annot=True, fmt='f', linewidths=.5, cmap='RdPu')

plt.title('범죄 발생(절도발생으로 정렬) - 각 항목별 최대값으로 나눠 정규화')
plt.show()

#### (단순히 범죄건수만 보지 말고) 인구수로 나눠서 인구대비 발생비율로 살펴보기

In [ ]:
crime_count_norm.head(3)

In [ ]:
gu_df.head(3)

In [ ]:
# 행(구)별로 구별 범죄 수 (max 대비 비율값) / 구별 인구 수 * 100000 
# 인구 수 단위인 10만을 곱해준다 (강서구 강간 = 9.795665e-07 -> 0.x 까지 끌어올리기)

crime_ratio = crime_count_norm.div(gu_df['인구수'], axis=0) * 100000 
crime_ratio.head()

#### [ 인구수 대비 ]  구별 살인 발생 순위 살펴보기

In [ ]:
plt.figure(figsize = (10,10))

sns.heatmap(crime_ratio.sort_values(by='살인', ascending=False), annot=True, fmt='f', linewidths=.5, cmap='Reds')
plt.title('범죄 발생(살인발생으로 정렬) - 각 항목을 정규화한 후 인구로 나눔')
plt.show()

#### [ 인구 수 대비 ] 구별 5대범죄 발생 수치 평균

In [ ]:
# 구별 인구 대비 
crime_ratio['전체발생비율'] = crime_ratio.mean(axis=1) # axis=1 : columns-oriented
crime_ratio.head()

#### [ 인구 수 대비 ] [ 5대범죄 발생 수치 평균 ] 기준 구별 순위 비교

In [ ]:
plt.figure(figsize = (10,10))

sns.heatmap(crime_ratio.sort_values(by='전체발생비율', ascending=False), annot=True, fmt='f', linewidths=.5, cmap='Reds')
plt.title('범죄 발생(전체발생비율로 정렬) - 각 항목을 정규화한 후 인구로 나눔')
plt.show()

<br>
#### 3. 데이터 시각화 (Data visualization, geo-mapping)

- 지도 시각화 : Folium library 을 활용합니다. 
- 지도 데이터 : https://github.com/southkorea/southkorea-maps 에서 서울만 따로 추린 GeoJSON 데이터를 활용합니다
  <br>(southkorea-maps/kostat/2013/json/skorea_municipalities_geo_simple.json)
<br>
<br>
- GeoJSON : JSON 데이터 형식을 활용한 공간 데이터 교환 포맷(Geospatial Data Interchange Format)
- GeoJSON에서 Feature는 Geometry object와 속성정보를 담고 있고, Feature 컬렉션은 Feature의 집합으로 구성됩니다.
- 상세 정보 링크 : https://goo.gl/GL2F2w & https://goo.gl/E4NCLC
<br>
<br>
- ** Folium library 설치 : **
- pip install folium
- pip install --index-url=http://pypi.python.org/simple/ --trusted-host pypi.python.org folium
- conda config --set ssl_verify false 입력 후 -> conda install folium

![image.png](attachment:image.png)

In [ ]:
# import warnings
# warnings.simplefilter(action = "ignore", category = FutureWarning)

import json

geo_path = 'skorea_municipalities_geo_simple.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))

In [ ]:
geo_str['features'][0]

In [ ]:
geo_str['features'][0]['id']

#### 구별 살인사건 발생 건수 시각화

In [ ]:
import folium

# tiles = 지도 타입 (Stamen Terrain 도 가능)
# location : 초기 지도 center 위치
map = folium.Map(location=[37.5502, 126.982], zoom_start=11, tiles='Stamen Toner')

# 살인사건 발생건수 시각화
# choropleth : Choropleth map @ https://goo.gl/yrTRHU
map.choropleth(geo_data = geo_str, # 서울시 행정구역별 polygon drawing
               data = gu_df['살인'], # 시각화의 대상이 될 데이터
               columns = [gu_df.index, gu_df['살인']], # 1) df의 index 칼럼을 가져와 인식하고
               fill_color = 'PuRd', #PuRd, YlGnBu (maybe Purple & Red)
               key_on = 'feature.id') # GeoJSON 규약을 따름, json 파일(지도 데이터)의 "feature" type의 "id" 에 매칭된다

# key_on: Variable in the GeoJSON file to bind the data to. 
# Must always start with 'feature' and be in JavaScript objection notation. 
# Ex: 'feature.id' or 'feature.properties.statename'.

map

#### [ 인구 수 대비 ] [ 5대범죄 발생 수치 평균 ] 기준 구별 데이터 시각화

In [ ]:
map = folium.Map(location=[37.5502, 126.982], zoom_start=11, tiles='Stamen Toner')

# 전체 5대 범죄 인구당 발생비율 시각화
map.choropleth(geo_data = geo_str,
               data = crime_ratio['전체발생비율'],
               columns = [crime_ratio.index, crime_ratio['전체발생비율']],
               fill_color = 'PuRd', #PuRd, YlGnBu
               key_on = 'feature.id')
map

#### 구별 검거율 데이터 시각화

In [ ]:
map = folium.Map(location=[37.5502, 126.982], zoom_start=11, tiles='Stamen Toner')

# 검거율 시각화
map.choropleth(geo_data= geo_str,
               data = gu_df['검거율'],
               columns = [gu_df.index, gu_df['검거율']],
               fill_color = 'YlGnBu', #PuRd, YlGnBu (Yellow, Green, Blue)
               key_on = 'feature.id')
map

In [ ]:
# Saving a folium map as an HTML file

# map.save('folium_map.html')

In [ ]:
# DF to csv file

# gu_df.to_csv('processed_data.csv', encoding='utf-8') # 혹은 euc-kr or cp949

In [ ]:
# DF to Excel file

# from pandas import ExcelWriter
# writer = ExcelWriter('file_name.xlsx')
# gu_df.to_excel(writer)
# writer.save()